In [1]:
import urllib
import ntpath
from pipelineapi import PipelineAPI
import os
import json
import glob


def path_leaf(path):
    if urllib.parse.urlparse(path).scheme != "" or os.path.isabs(path):
        path = os.path.split(path)[-1]
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

In [2]:
import ast

anchors_pth = '/media/dean/datastore/nebula/pipelines/nets/darknet/anchors.txt'
with open(anchors_pth) as f:
    anchors_str = f.readlines()[0].strip('\'').strip('\t').strip()

# Save anchors in trn_model_cfg
anchor_list = list(ast.literal_eval(anchors_str))
it = iter(anchor_list)
anchs =  zip(*[iter(anchor_list)]*2)

print(list(anchs))

FileNotFoundError: [Errno 2] No such file or directory: '/media/dean/datastore/nebula/pipelines/nets/darknet/anchors.txt'

In [72]:
from pipelineapi import PipelineAPI
import os

def dataset_import(dataset_name, dataset_file):
        data_pipeline_api = PipelineAPI("http://ec2-54-184-229-238.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')
        with open(dataset_file, 'rb') as f:
            data_pipeline_api.upload_json_file(f, file_name=path_leaf(dataset_file), round='inspection', dataset_name=dataset_name, skip_image=False)
            

In [73]:
def update_cats(data):
    for img_data in data:
        if img_data.get('labels', None):
            deletions = []
            for idx, label in enumerate(img_data['labels']):
                # If detected - delete
                if 'detected-' in label['category']:
                    label['category'] = label['category'].replace('detected-', '')
                if 'tracked-' in label['category']:
                    deletions.append(label)
                if 'flickering_' in label['category']:
                    label['category'] = label['category'].replace('flickering_', '')
            
            for label in deletions:
                img_data['labels'].remove(label)
                
    return data

In [74]:
def strip_kache_data(data):
    for img_data in data:
        img_data['kache_id'] = None
        if img_data.get('labels', None):
            for idx, label in enumerate(img_data['labels']):
                    label['kache_label_id'] = None
        elif img_data.get('dataset_name', None):
            img_data['dataset_name'] = None
    return data

In [75]:
def get_img_data(data):
    for img_data in data:
        found = False
        # Find corresponding img_data dict
        for dset in glob.glob('/media/dean/datastore/datasets/070219_trn/*.json'):

            with open(dset, 'r') as fn:
                img_data_list = json.load(fn)
                
            for db_img_data in img_data_list:  
        
                if img_data['url'] == db_img_data.get('url', None):
                    # copy over data
                    #print(f"Url found: {img_data['url']}")
                    img_data['attributes'] = db_img_data.get('attributes', None)
                    img_data['videoName'] = db_img_data.get('videoName', None)
                    img_data['timestamp'] = db_img_data.get('timestamp', None)
                    img_data['latitude'] = db_img_data.get('latitude', None)
                    img_data['longitude'] = db_img_data.get('longitude', None)
                    img_data['raw_timestamp'] = db_img_data.get('raw_timestamp', None)
                    img_data['hash'] = os.path.splitext(path_leaf(db_img_data.get('url', None)))[0]
                    img_data['index'] = db_img_data.get('index', None)
                    img_data['width'] = db_img_data.get('width', None)
                    img_data['height'] = db_img_data.get('height', None)
                    
                    
                    
                    # break out of loop
                    found = True
                    break
            if found: break
    return data

In [76]:
def strip_labels(data):
    for img_data in data:
        if img_data.get('labels', None):
            img_data['labels'] = []
    return data

In [77]:
def update_traffic_light_cats(data):
    for img_data in data:
        if img_data.get('labels', None):
            for label in [l for l in  img_data['labels'] if l['category'] == 'traffic light' and l.get('attributes', None)]:
                if label['attributes'].get('trafficLightColor', None):
                    if label['attributes']['trafficLightColor'][1].lower() == 'green' or label['attributes']['trafficLightColor'][1].lower() == 'g'  or label['attributes']['trafficLightColor'][0] == 1:
                        label['category'] = 'traffic light-green'
                    elif label['attributes']['trafficLightColor'][1].lower() == 'yellow' or label['attributes']['trafficLightColor'][1].lower() == 'y'  or label['attributes']['trafficLightColor'][0] == 2:
                        label['category'] = 'traffic light-amber'
                    elif label['attributes']['trafficLightColor'][1].lower() == 'red' or label['attributes']['trafficLightColor'][1].lower() == 'r'  or label['attributes']['trafficLightColor'][0] == 3:
                        label['category'] = 'traffic light-red'
                    else:
                        label['category'] = 'traffic light'

                # Support both old and new bdd formats
                elif label['attributes'].get('Traffic Light Color', None):
                    if label['attributes']['Traffic Light Color'][1].lower() == 'g' or label['attributes']['Traffic Light Color'][0] == 1:
                        label['category'] = 'traffic light-green'
                    elif label['attributes']['Traffic Light Color'][1].lower() == 'y' or label['attributes']['Traffic Light Color'][0] == 2:
                        label['category'] = 'traffic light-amber'
                    elif label['attributes']['Traffic Light Color'][1].lower() == 'r' or label['attributes']['Traffic Light Color'][0] == 3:
                        label['category'] = 'traffic light-red'
                    else:
                        label['category'] = 'traffic light'
    return data

In [78]:
def prune_labels(data):
    for img_data in data:
        if img_data.get('labels', None):
            tmp = []
    
            for label in img_data['labels']:
                if label.get('label_status', None):
                    del label['label_status']
                    tmp.append(label)
            img_data['labels'] = tmp
    return data
                    

In [ ]:
import glob
import random
import glob
import json
import numpy as np
import hashlib
import toml
import shutil
import subprocess
from subprocess import Popen,PIPE,STDOUT,call
import json

anns_dir = '/media/kuser/datastore/nebula/pipelines/kache/frames/external_sources/'
new_dir = '/media/kuser/datastore/complete_annotations/'

os.makedirs(new_dir, exist_ok = True)
for dset in glob.glob(os.path.join(anns_dir, '**', '*flickers.json')):
    with open(dset, 'r') as fn:
        data = json.load(fn)
        

    data = update_cats(data)
    data = strip_kache_data(data)
    data = update_traffic_light_cats(data)
    data = prune_labels(data)
    
    with open(os.path.join(new_dir, path_leaf(dset)), 'w+') as fn:
        json.dump(data, fn)

In [ ]:
import glob
import random
import glob
import json
import numpy as np
import hashlib
import toml
import shutil
import subprocess
from subprocess import Popen,PIPE,STDOUT,call
import json

check_s3 = True
def send_to_s3( img_path, s3_bucket = 'kache-scalabel/kache_ai/frames/'):
    s3_path = os.path.join(s3_bucket, path_leaf(img_path))
    s3_bucket = 's3://'+s3_bucket
    sp = subprocess.Popen("aws s3 cp {} {}".format(img_path, s3_bucket), shell=True, stdout=PIPE)
    out_str = sp.communicate()
    return os.path.join('https://s3-us-west-2.amazonaws.com', s3_path)

In [ ]:
import glob
import json

anns_dir = '/media/kuser/datastore/nebula/pipelines/kache/frames/external_sources/'
new_dir = '/media/kuser/datastore/complete_annotations/'
dataset_files = glob.glob(os.path.join(new_dir, '*.json'))

# Change name/url to s3_url
for idx, ann_file in enumerate(dataset_files):
    with open(ann_file, 'r') as f:
        dataset = json.load(f)

    for img_data in dataset:
        hex_val = os.path.abspath(img_data['url']).replace('/', '^^')
        hash_object = hashlib.sha1(str.encode(hex_val))
        hex_dig = hash_object.hexdigest()

        tmp_img_dir = os.path.join(anns_dir, 'images')
        os.makedirs(tmp_img_dir, exist_ok = True)
        orig_frame_path = img_data['url']
        with open('/media/kuser/datastore/nebula/pipelines/configs/entanglement_cfg.tml', 'r') as tmlfile:
            cfg_data = toml.load(tmlfile)

        orig_frame_path = os.path.join(cfg_data['entanglement']['master_nebula_dir'], img_data['url'].split('nebula')[-1].strip('/') )
        new_path = os.path.join(tmp_img_dir, str(hex_dig)+ os.path.splitext(path_leaf(img_data['url']))[-1] )

        shutil.copyfile(orig_frame_path, new_path)
        s3uri = send_to_s3( new_path )

        img_data['url'] = s3uri



    with open(ann_file, 'w+') as f:
        json.dump(dataset, f)
    

In [ ]:
for file in glob.glob(os.path.join(new_dir,'*.json')):
    print(f'Importing file to db: {file}')
    dataset_name = 'mining_set'

    # dataset_import(dataset_name, file)

In [ ]:
import urllib
import ntpath
import json
import os

def get_dataset():
    data_pipeline_api = PipelineAPI("http://ec2-54-149-236-75.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')

    batch_size = 0
    batch = 0
    batch_anns = []


    dataset_names = ['062019', '032019']
    kache_anns_dir =  '/home/dean/Desktop'

    databunch = data_pipeline_api.data(dataset_names=dataset_names, video_source_names=[], categories_names=[],
                    label_statuses=[10], page_size=None, frame_id=None) # returns generator

    for anns in databunch:
#           anns = update_traffic_light_cats([anns])
        batch_anns.append(anns)
        batch_size += len(anns)
        if batch_size % 5000 == 0:
            with open(os.path.join(kache_anns_dir,f'{dataset_names}_{batch}--inspected.json'), 'w') as fn:
                json.dump(batch_anns, fn)
            batch_anns = []
            batch += 1

    # Save any remaining frames
    if len(batch_anns) != 0:
        with open(os.path.join(kache_anns_dir,f'{dataset_names}_{batch}--inspected.json'), 'w') as fn:
            json.dump(batch_anns, fn)


In [ ]:
import urllib
import ntpath
import json
import os

def get_dataset():
    data_pipeline_api = PipelineAPI("http://ec2-54-149-236-75.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')

    batch_size = 0
    batch = 0
    batch_anns = []


    dataset_names = ['062019', '032019']
    kache_anns_dir =  '/home/dean/Desktop'

    databunch = data_pipeline_api.data(dataset_names=dataset_names, video_source_names=[], categories_names=['motor','motorbike','motorcycle'],
                    label_statuses=[], page_size=None, frame_id=None) # returns generator

    for anns in databunch:
#           anns = update_traffic_light_cats([anns])
        batch_anns.append(anns)
        batch_size += len(anns)
        if batch_size % 25000 == 0:
            with open(os.path.join(kache_anns_dir,f'{"^".join(dataset_names)}_{batch}--motorcycles.json'), 'w') as fn:
                json.dump(batch_anns, fn)
            batch_anns = []
            batch += 1

    # Save any remaining frames
    if len(batch_anns) != 0:
        with open(os.path.join(kache_anns_dir,f'{"^".join(dataset_names)}_{batch}--motorcycles.json'), 'w') as fn:
            json.dump(batch_anns, fn)


In [ ]:
get_dataset()

In [ ]:
def get_dataset():
    data_pipeline_api = PipelineAPI("http://ec2-54-191-65-138.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')

    batch_size = 0
    batch = 0
    batch_anns = []


    dataset_names = 'china_set'
    kache_anns_dir =  '/home/dean/Desktop'

    databunch = data_pipeline_api.data(dataset_names=dataset_names, video_source_names=[], categories_names=[],
                    label_statuses=[20], page_size=None, frame_id=None) # returns generator

    for anns in databunch:
#             anns = update_traffic_light_cats([anns])
        batch_anns.append(anns)
        batch_size += len(anns)
        if batch_size % 5000 == 0:
            with open(os.path.join(kache_anns_dir,f'{dataset_names}_{batch}--rejected.json'), 'w') as fn:
                json.dump(batch_anns, fn)
            batch_anns = []
            batch += 1

    # Save any remaining frames
    if len(batch_anns) != 0:
        with open(os.path.join(kache_anns_dir,f'{dataset_names}_{batch}--rejected.json'), 'w') as fn:
            json.dump(batch_anns, fn)


In [ ]:
get_dataset()

In [ ]:
from collections import defaultdict
import numpy as np

attr2idx = {}
idx2attr = {}

In [ ]:
attr2idx['video_source'] = 0
attr2idx['cam_num'] = 1
attr2idx['latitude'] = 2
attr2idx['longitude'] = 3
attr2idx['hour_of_day'] = 4

# Create vector that stores a representation of the object distribution from the log, sorted by class, then type, then box size (in descending order)
classes = ["bike", "bus", "car", "construct-equipment", "construct-pole", "construct-post", "construct-sign", "motorcycle", "person", "rider", "traffic light", "traffic light-amber", "traffic light-green", "traffic light-red", "traffic sign", "traffic sign-slow_sign", "traffic sign-speed_sign", "traffic sign-stop_sign", "trailer", "train", "truck",]
#
idx = 5
for cls in classes:
    for tracking_type in ['detected', 'tracked', 'flickering_tracked']:
        print(f'total_{tracking_type}-{cls}',':',idx)
        attr2idx[f'total_{tracking_type}-{cls}'] = idx
        idx += 1

for cls in classes:
    for tracking_type in ['detected', 'tracked', 'flickering_tracked']:
        # Load the first 10 objects in each category
        for i in np.arange(1, 11):
            
            #x1 
            print(f'{tracking_type}-{cls}_{i}_x1',':',idx)
            attr2idx[f'{tracking_type}-{cls}_{i}_x1'] = idx
            idx += 1
            
            #y1 
            print(f'{tracking_type}-{cls}_{i}_y1',':',idx)
            attr2idx[f'{tracking_type}-{cls}_{i}_y1'] = idx
            idx += 1
            
            #x2 
            print(f'{tracking_type}-{cls}_{i}_x2',':',idx)
            attr2idx[f'{tracking_type}-{cls}_{i}_x2'] = idx
            idx += 1
            
            #y2 
            print(f'{tracking_type}-{cls}_{i}_y2',':',idx)
            attr2idx[f'{tracking_type}-{cls}_{i}_y2'] = idx
            idx += 1

idx2attr = {v_i: v for v, v_i in attr2idx.items()}          
    

In [ ]:
print(idx2attr)

In [ ]:
a = dict.fromkeys(attr2idx.keys(),0)
print(a)

In [ ]:
import os
import glob
import datetime
import ast
import json

run_eval = True
if run_eval:
    with open('/home/kuser/Desktop/master_log_features_062419.json', 'w+') as fn:
        json_str = json.dumps([])
        fn.write(json_str)
        
    for flickers in glob.glob(os.path.join('/media/kuser/Data/','06242019_trn','*.json')):
        print(flickers)
        # Collect attributes
        with open(flickers, 'r') as fn:
            flicker_data = json.loads(fn.read())

        if flicker_data and flicker_data[0].get('videoName', None) and 'unknown' not in flicker_data[0]['videoName'].lower():
            feat_vec = dict.fromkeys(attr2idx.keys(), '0.0')
            if 'cam' in flicker_data[0]['videoName']:
                cam_num = int(float(flicker_data[0]['videoName'].split('cam')[-1]))
                feat_vec["cam_num"] = str(cam_num)


            with open('/home/kuser/Desktop/master_log_features_062419.json', 'r') as fn:
                master_feats = json.load(fn)



            # Generate vector
            feat_vec["video_source"] = flicker_data[0]['videoName']


            if flicker_data[0].get('latitude', None):    
                feat_vec["latitude"] = str(flicker_data[0]['latitude'])

            if flicker_data[0].get('longitude', None):    
                feat_vec["longitude"] = str(flicker_data[0]['longitude'])

            if flicker_data[0].get('timestamp', None):
                timestamp = int(flicker_data[0]['timestamp'])
                timestamp_s = timestamp/1e9

                hour = datetime.datetime.fromtimestamp(timestamp_s).strftime('%H')            
                feat_vec["hour_of_day"] = hour



            for flicker_frame in flicker_data:
                flicker_frame_detections = []
                for label in flicker_frame.get('labels'):
                    feat_vec[f"total_{label['category']}"] = str( int( float(feat_vec[f"total_{label['category']}"])) +1 )

                    label_area =  np.abs((label['box2d']['x2'])-(label['box2d']['x1']) * (label['box2d']['y2'])-(label['box2d']['y1']))                
                    flicker_frame_detections.append([f"{label['category']}", label_area, [label["box2d"]["x1"], label["box2d"]["y1"],label["box2d"]["x2"], label["box2d"]["y2"]]] )

                # Fill in detections
                for cls in classes:
                    for tracking_type in ['detected', 'tracked', 'flickering_tracked']:

                        class_dets = [det for det in flicker_frame_detections if f'{tracking_type}-{cls}' in det[0]]

                        # Sort class_dets by area size
                        sorted_class_dets = (sorted(class_dets, key = lambda x: x[1]))

                        # Only process 10 object maximum
                        while len(sorted_class_dets) <= 10:
                            sorted_class_dets.append([f'{tracking_type}-{cls}', 0.0, [0.0, 0.0, 0.0, 0.0]])

                        if len(sorted_class_dets) > 10:
                             sorted_class_dets = sorted_class_dets[:10]

                        # Load the first 10 objects in each category
                        for i, det in enumerate(sorted_class_dets, 1):
                            #x1 
                            feat_vec[f"{tracking_type}-{cls}_{i}_x1"] = str(det[2][0])

                            #y1 
                            feat_vec[f"{tracking_type}-{cls}_{i}_y1"] = str(det[2][1])

                            #x2 
                            feat_vec[f"{tracking_type}-{cls}_{i}_x2"] = str(det[2][2])

                            #y2 
                            feat_vec[f"{tracking_type}-{cls}_{i}_y2"] = str(det[2][3])



            flicker_data[0]['feature_vector'] = feat_vec
            master_feats.append(feat_vec)        







            # Save vector in json file
            with open(flickers, 'w+') as fn:
                json.dump(flicker_data, fn)  
            # Save vector in json file
            with open('/home/kuser/Desktop/master_log_features_062419.json', 'w+') as fn:
                json_str = json.dumps( master_feats )
                fn.write(json_str)



        else:
            print(f'Skipping {flickers}')

In [90]:
with open('/home/kuser/Desktop/master_log_features_062419.json', 'r') as fn:
    master_feats =  json.load(fn)
    log_source_names = [list(v.values())[0] for idx,v in enumerate(master_feats)] 
    X = np.array([list(v.values())[1:] for idx,v in enumerate(master_feats)], dtype=np.float32)
    
    

In [91]:
print(X) ; print(X.shape); print(log_source_names)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(5181, 2587)
['1003-frames', '1010-frames', '1020-frames', '1025-frames', '1028-frames', '1031-frames', '1033-frames', '1049-frames', '1057-frames', '1061-frames', '1063-frames', '1082-frames', 'bagfile_18-11-18_2137_8', 'bagfile_18-11-18_2218_112', 'bagfile_18-11-18_2218_11', 'bagfile_18-11-18_2218_22', 'bagfile_18-11-18_2218_32', 'bagfile_18-11-18_2218_33', 'bagfile_18-11-18_2218_34', 'bagfile_18-10-26_0806_8', 'bagfile_18-10-26_0806_9', 'bagfile_18-10-26_0857_0', 'bagfile_18-10-26_1050_0', 'bagfile_18-10-26_1120_10', 'bagfile_18-10-26_1120_113', '1092-frames', '1234-frames', '1341-frames', '1405-frames', '1690-frames', '1746-frames', '1889-frames', 'bagfile_18-12-26_1153_1', 'bagfile_18-12-26_1153_2', 'bagfile_18-12-26_1153_3', 'bagfile_18-12-26_1153_4', 'bagfile_18-12-26_1153_5', 'bagfile_18-12-26_1153_6', 'bagfile_18-12-26_1153

In [92]:
from sklearn.cluster import MiniBatchKMeans
from sklearn import preprocessing

mm_scaler = preprocessing.MinMaxScaler()
X_scaled = mm_scaler.fit_transform(X)

kmeans = MiniBatchKMeans(n_clusters=30,
                          random_state=0,
                          batch_size=10,
                          max_iter=100).fit(X_scaled)


In [93]:
print(kmeans.cluster_centers_)
print(len(kmeans.cluster_centers_))

[[0.2582417  0.37779382 0.42294717 ... 0.         0.         0.        ]
 [0.11949688 0.44280103 0.37318647 ... 0.         0.         0.        ]
 [0.2026239  0.5120082  0.28894395 ... 0.         0.         0.        ]
 ...
 [0.19496855 0.3997745  0.4065017  ... 0.         0.         0.        ]
 [0.16975309 0.43783933 0.3697029  ... 0.         0.         0.        ]
 [0.2676057  0.4239094  0.35125118 ... 0.         0.         0.        ]]
30


In [96]:
groups = kmeans.predict(X_scaled)

In [97]:
clustered_logs = []

for idx, group in enumerate(groups):
    print(f"log: {log_source_names[idx]} | cluster: {group}")
    clustered_logs.append((log_source_names[idx], group))

log: 1003-frames | cluster: 4
log: 1010-frames | cluster: 1
log: 1020-frames | cluster: 9
log: 1025-frames | cluster: 3
log: 1028-frames | cluster: 21
log: 1031-frames | cluster: 19
log: 1033-frames | cluster: 4
log: 1049-frames | cluster: 3
log: 1057-frames | cluster: 16
log: 1061-frames | cluster: 16
log: 1063-frames | cluster: 28
log: 1082-frames | cluster: 20
log: bagfile_18-11-18_2137_8 | cluster: 14
log: bagfile_18-11-18_2218_112 | cluster: 20
log: bagfile_18-11-18_2218_11 | cluster: 28
log: bagfile_18-11-18_2218_22 | cluster: 22
log: bagfile_18-11-18_2218_32 | cluster: 28
log: bagfile_18-11-18_2218_33 | cluster: 14
log: bagfile_18-11-18_2218_34 | cluster: 7
log: bagfile_18-10-26_0806_8 | cluster: 3
log: bagfile_18-10-26_0806_9 | cluster: 0
log: bagfile_18-10-26_0857_0 | cluster: 10
log: bagfile_18-10-26_1050_0 | cluster: 8
log: bagfile_18-10-26_1120_10 | cluster: 26
log: bagfile_18-10-26_1120_113 | cluster: 26
log: 1092-frames | cluster: 7
log: 1234-frames | cluster: 3
log: 1341

log: alanturing-2019-04-18--18-27-42--cam1 | cluster: 8
log: alanturing-2019-04-18--18-37-42--cam0 | cluster: 3
log: alanturing-2019-04-18--18-39-42 | cluster: 18
log: alanturing-2019-04-18--18-47-42--cam2 | cluster: 20
log: alanturing-2019-04-18--18-52-42--cam0 | cluster: 5
log: alanturing-2019-04-18--19-00-42--cam1 | cluster: 18
log: alanturing-2019-04-18--19-01-42--cam1 | cluster: 1
log: alanturing-2019-04-19--07-32-30--cam1 | cluster: 27
log: alanturing-2019-04-19--07-33-30--cam0 | cluster: 6
log: alanturing-2019-04-19--07-33-30--cam1 | cluster: 3
log: alanturing-2019-05-01--15-57-32--cam0 | cluster: 22
log: alanturing-2019-05-01--16-58-40--cam0 | cluster: 15
log: alanturing-2019-05-13--05-20-40--cam1 | cluster: 23
log: bagfile_18-08-30_1152_7 | cluster: 21
log: bagfile_18-09-18_0905_112 | cluster: 2
log: alanturing-2019-02-04--11-24-02--cam0 | cluster: 7
log: alanturing-2019-02-04--11-34-02 | cluster: 22
log: alanturing-2019-02-04--11-42-02--cam0 | cluster: 22
log: alanturing-2019

In [98]:
## Workflow for Quasars

## Supernovae Generates Flickers JSON and pushed message to generate feature vector ##

## Download log from flickering db (get all frames under same video_source_name) ##
## Generate Feature Vector ##
## SDave Feature Vector in Cached Json Flickers File ##

In [99]:
def get_best_log(cluster_matrix, category, attr2idx, log_feat_vectors):
    best_log = None
    best_flickering_cat_element = np.amax(cluster_matrix[:, attr2idx[f'total_flickering_tracked-{category}'] - 1])
    best_tracked_cat_element = np.amax(cluster_matrix[:, attr2idx[f'total_tracked-{category}'] - 1])
    best_detected_cat_element = np.amax(cluster_matrix[:, attr2idx[f'total_detected-{category}'] - 1])
    
    if best_flickering_cat_element > 0:
        print("best_flickering_cat_element", best_flickering_cat_element)
        best_flickering_cat_log_idx = np.where( cluster_matrix[:, attr2idx[f'total_flickering_tracked-{category}'] - 1] == best_flickering_cat_element)[0]

        if type(best_flickering_cat_log_idx) != int:
            if best_flickering_cat_log_idx.any():
                best_flickering_cat_log_idx = best_flickering_cat_log_idx.ravel()[0]
                best_flickering_cat_log_idx = np.asscalar(best_flickering_cat_log_idx)
                
                
                print("best_flickering_cat_log_idx", best_flickering_cat_log_idx)
                best_log_vec = X[best_flickering_cat_log_idx]
                best_log_list = [log for log, vec in log_feat_vectors.items() if np.array_equal(best_log_vec, vec)]
                
                if any(best_log_list):
                    best_log = best_log_list[0]
                
                
#                 best_log = list(log_feat_vectors.keys())[best_flickering_cat_log_idx]
                print("BEST FLICKERING LOG: ", best_log, cluster_matrix[best_flickering_cat_log_idx, attr2idx[f'total_flickering_tracked-{category}'] - 1])
            

    elif best_tracked_cat_element > 0:
        print("best_tracked_cat_element", best_tracked_cat_element)
        best_tracked_cat_log_idx = np.where( cluster_matrix[:, attr2idx[f'total_tracked-{category}'] - 1] == best_tracked_cat_element)[0]

        if type(best_tracked_cat_log_idx) != int:
            if best_tracked_cat_log_idx.any():
                best_tracked_cat_log_idx = best_tracked_cat_log_idx.ravel()[0]
                best_tracked_cat_log_idx = np.asscalar(best_tracked_cat_log_idx)

                print("best_tracked_cat_log_idx", best_tracked_cat_log_idx)
                best_log_vec = X[best_tracked_cat_log_idx]
                best_log_list = [log for log, vec in log_feat_vectors.items() if np.array_equal(best_log_vec, vec)]
                
                if any(best_log_list):
                    best_log = best_log_list[0]
#                 best_log = list(log_feat_vectors.keys())[int(best_tracked_cat_log_idx)]
                print("BEST TRACKED LOG: ", best_log, (cluster_matrix[best_tracked_cat_log_idx, attr2idx[f'total_tracked-{category}'] - 1]))
        
    elif best_detected_cat_element > 0:
        print("best_detected_cat_element", best_detected_cat_element)
        best_detected_cat_log_idx = np.where( cluster_matrix[:, attr2idx[f'total_tracked-{category}'] - 1] == best_detected_cat_element)[0]

        if type(best_detected_cat_log_idx) != int:
            if best_detected_cat_log_idx.any():
                best_detected_cat_log_idx = best_detected_cat_log_idx.ravel()[0]
                best_detected_cat_log_idx = np.asscalar(best_detected_cat_log_idx)
            
                print("best_detected_cat_log_idx", best_detected_cat_log_idx)
                best_log_vec = X[best_detected_cat_log_idx]
                best_log_list = [log for log, vec in log_feat_vectors.items() if np.array_equal(best_log_vec, vec)]
                
                if any(best_log_list):
                    best_log = best_log_list[0]
                
                
#                 best_log = list(log_feat_vectors.keys())[int(best_detected_cat_log_idx)]
                print("BEST DETECTED LOG: ", best_log, (cluster_matrix[best_detected_cat_log_idx, attr2idx[f'total_tracked-{category}'] - 1]))
        
    else:
         print(f"No Best log found for category: {category}.. Skipping")
            
          
    return best_log

In [100]:
from collections import OrderedDict 

log_cluster = defaultdict(list)

for log in clustered_logs:
    log_cluster[log[1]].append(log[0])
    
logs_selected = []
labels  = [ "bike", "bus", "car", "construct-equipment", "construct-pole", "construct-post", "construct-sign", "motorcycle", "person", "rider", "traffic light", "traffic light-amber", "traffic light-green", "traffic light-red", "traffic sign", "traffic sign-slow_sign", "traffic sign-speed_sign", "traffic sign-stop_sign", "trailer", "train", "truck",]
with open('/home/kuser/Desktop/master_log_features_062419.json', 'r') as fn:
    master_feats =  json.load(fn)
        
for cluster, logs in log_cluster.items():
    # Generate table of logs and feature vectors for each cluster
    log_feat_vectors = OrderedDict()
    
    # sort logs by most flickers of category
    # select log with most
    print("\n\nCluster:", cluster)
    
    for log in sorted(logs):
#         with open(glob.glob(os.path.join(f"/home/dean/Desktop/flickers/{log}*"))[0], 'r') as fn:
#             flicker_data = json.loads(fn.read())
        
            
        log_idx = [idx for idx, feat in enumerate(master_feats) if feat['video_source'] == log][0]
        log_feat_vectors[log] = np.array(list(master_feats[log_idx].values())[1:], dtype= np.float32)
        

    daytime_delta = [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17 ,18, 19]
    for cat in labels:
        # Get best daytime log for category
        X = np.array([v for k, v in log_feat_vectors.items() if k not in logs_selected and v[attr2idx['hour_of_day']] in daytime_delta])
        if X.any():
            best_log = get_best_log(X, cat, attr2idx, log_feat_vectors)
            if best_log: 
                logs_selected.append(best_log)
                
        # Get best nighttime log for category
        X = np.array([v for k, v in log_feat_vectors.items() if k not in logs_selected and v[attr2idx['hour_of_day']] not in daytime_delta])
        if X.any():
            best_log = get_best_log(X, cat, attr2idx, log_feat_vectors)
            if best_log: 
                logs_selected.append(best_log)
    
print(set(logs_selected))   
print(len(set(logs_selected)))



Cluster: 4
best_tracked_cat_element 9.0
best_flickering_cat_element 572.0
best_flickering_cat_log_idx 126
BEST FLICKERING LOG:  chrysippus-2019-01-06--13-59-12--cam1 572.0
best_flickering_cat_element 14.0
best_flickering_cat_element 480.0
best_flickering_cat_log_idx 136
BEST FLICKERING LOG:  chrysippus-2019-03-31--19-38-42--cam0 480.0
best_flickering_cat_element 93.0
best_flickering_cat_element 1977.0
best_flickering_cat_log_idx 172
BEST FLICKERING LOG:  lily-2019-04-17--17-37-04--cam0 1977.0
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: construct-post.. Skipping
No Best log found for category: construct-post.. Skipping
No Best log found for category: construct-sign.. Skipping
No Best log found for category: construct-sign.. Skipping
No Best log found for cate

BEST FLICKERING LOG:  bagfile_19-01-04_1217_58 74.0
best_tracked_cat_element 71.0
best_tracked_cat_element 3119.0
best_tracked_cat_log_idx 189
BEST TRACKED LOG:  bagfile_19-01-09_1122_15 3119.0
best_flickering_cat_element 2915.0
best_flickering_cat_log_idx 4
BEST FLICKERING LOG:  bagfile_18-10-24_1129_9 2915.0
best_flickering_cat_element 2249.0
best_flickering_cat_log_idx 128
BEST FLICKERING LOG:  bagfile_18-10-24_1129_10 2249.0
No Best log found for category: traffic sign-slow_sign.. Skipping
No Best log found for category: traffic sign-slow_sign.. Skipping
No Best log found for category: traffic sign-speed_sign.. Skipping
No Best log found for category: traffic sign-speed_sign.. Skipping
No Best log found for category: traffic sign-stop_sign.. Skipping
No Best log found for category: traffic sign-stop_sign.. Skipping
No Best log found for category: trailer.. Skipping
No Best log found for category: trailer.. Skipping
No Best log found for category: train.. Skipping
No Best log found 

best_flickering_cat_element 629.0
best_flickering_cat_log_idx 2
BEST FLICKERING LOG:  alanturing-2019-03-26--18-42-10--cam1 629.0
best_flickering_cat_element 79.0
best_flickering_cat_log_idx 6
BEST FLICKERING LOG:  299-frames 79.0
best_tracked_cat_element 18.0
best_tracked_cat_log_idx 3
BEST TRACKED LOG:  chrysippus-2019-04-03--22-01-35 18.0
best_flickering_cat_element 101.0
best_flickering_cat_log_idx 175
BEST FLICKERING LOG:  chrysippus-2019-03-19--16-14-35--cam0 101.0
best_flickering_cat_element 480.0
best_flickering_cat_log_idx 1
BEST FLICKERING LOG:  alanturing-2019-03-26--15-47-37 480.0
best_flickering_cat_element 5277.0
best_flickering_cat_log_idx 141
BEST FLICKERING LOG:  bagfile_19-01-11_0711_10 5277.0
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: const

BEST FLICKERING LOG:  bagfile_19-01-08_1341_32 3371.0


Cluster: 22
best_tracked_cat_element 12.0
best_tracked_cat_log_idx 5
BEST TRACKED LOG:  chrysippus-2019-03-31--17-58-05--cam0 12.0
best_flickering_cat_element 1139.0
best_flickering_cat_log_idx 133
BEST FLICKERING LOG:  bagfile_18-10-25_1349_1 1139.0
best_tracked_cat_element 5.0
best_flickering_cat_element 265.0
best_flickering_cat_log_idx 88
BEST FLICKERING LOG:  alanturing-2019-04-13--11-52-46--cam0 265.0
best_flickering_cat_element 2825.0
best_flickering_cat_log_idx 2
BEST FLICKERING LOG:  bagfile_19-01-07_1153_0 2825.0
best_flickering_cat_element 5361.0
best_flickering_cat_log_idx 164
BEST FLICKERING LOG:  bagfile_19-01-07_1530_0 5361.0
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: construct-post.. Skip

best_tracked_cat_element 9.0
best_tracked_cat_element 28.0
best_tracked_cat_log_idx 47
BEST TRACKED LOG:  bagfile_18-10-24_1704_11 28.0
best_tracked_cat_element 39.0
best_flickering_cat_element 150.0
best_flickering_cat_log_idx 69
BEST FLICKERING LOG:  chrysippus-2019-05-22--17-26-12--cam1 150.0
best_flickering_cat_element 19815.0
best_flickering_cat_log_idx 2
BEST FLICKERING LOG:  bagfile_18-11-15_1051_17 19815.0
best_flickering_cat_element 16020.0
best_flickering_cat_log_idx 56
BEST FLICKERING LOG:  bagfile_18-11-27_1123_22 16020.0
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: construct-post.. Skipping
No Best log found for category: construct-post.. Skipping
No Best log found for category: construct-sign.. Skipping
No Best log found for category: construct-si

best_tracked_cat_element 18.0
best_flickering_cat_element 682.0
best_flickering_cat_log_idx 11
BEST FLICKERING LOG:  883-frames 682.0
best_flickering_cat_element 34.0
best_flickering_cat_log_idx 4
BEST FLICKERING LOG:  bagfile_19-01-11_0711_24 34.0
best_flickering_cat_element 232.0
best_flickering_cat_log_idx 223
BEST FLICKERING LOG:  sallyride-2019-03-07--10-34-29--cam1 232.0
best_flickering_cat_element 1648.0
best_flickering_cat_log_idx 5
BEST FLICKERING LOG:  chrysippus-2019-03-31--13-49-54--cam0 1648.0
best_flickering_cat_element 7852.0
best_flickering_cat_log_idx 90
BEST FLICKERING LOG:  bagfile_18-10-06_1113_23 7852.0
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: construct-post.. Skipping
No Best log found for category: construct-post.. Skipping
No Best lo

best_flickering_cat_element 39.0
best_flickering_cat_log_idx 57
BEST FLICKERING LOG:  sallyride-2019-06-03--12-25-43--cam1 39.0
best_tracked_cat_element 15.0
best_flickering_cat_element 156.0
best_flickering_cat_log_idx 12
BEST FLICKERING LOG:  bagfile_18-10-26_1123_1 156.0
No Best log found for category: traffic light-amber.. Skipping
No Best log found for category: traffic light-amber.. Skipping
best_tracked_cat_element 832.0
best_flickering_cat_element 105.0
best_flickering_cat_log_idx 9
BEST FLICKERING LOG:  bagfile_18-10-06_1113_2 105.0
best_tracked_cat_element 459.0
best_flickering_cat_element 269.0
best_flickering_cat_log_idx 8
BEST FLICKERING LOG:  bagfile_18-09-30_1120_1 269.0
best_tracked_cat_element 733.0
best_flickering_cat_element 177.0
best_flickering_cat_log_idx 23
BEST FLICKERING LOG:  bagfile_18-12-14_1247_0 177.0
No Best log found for category: traffic sign-slow_sign.. Skipping
No Best log found for category: traffic sign-slow_sign.. Skipping
No Best log found for cat

best_tracked_cat_element 10.0
best_tracked_cat_log_idx 2
BEST TRACKED LOG:  1309-frames 10.0
best_tracked_cat_element 147.0
best_tracked_cat_log_idx 102
BEST TRACKED LOG:  bagfile_19-01-16_1017_14 147.0
No Best log found for category: traffic light-amber.. Skipping
best_tracked_cat_element 535.0
best_tracked_cat_log_idx 59
BEST TRACKED LOG:  bagfile_18-10-06_2235_40 535.0
best_tracked_cat_element 423.0
best_tracked_cat_log_idx 82
BEST TRACKED LOG:  bagfile_18-11-18_1707_0 423.0
best_flickering_cat_element 78.0
best_flickering_cat_log_idx 163
BEST FLICKERING LOG:  sallyride-2019-02-04--19-53-10--cam0 78.0
No Best log found for category: traffic sign-slow_sign.. Skipping
No Best log found for category: traffic sign-speed_sign.. Skipping
No Best log found for category: traffic sign-stop_sign.. Skipping
No Best log found for category: trailer.. Skipping
No Best log found for category: train.. Skipping
best_flickering_cat_element 65.0
best_flickering_cat_log_idx 148
BEST FLICKERING LOG:  ch

best_flickering_cat_element 391.0
best_flickering_cat_log_idx 4
BEST FLICKERING LOG:  373-frames 391.0
best_flickering_cat_element 261.0
best_flickering_cat_log_idx 19
BEST FLICKERING LOG:  alanturing-2019-02-21--14-25-14--cam1 261.0
best_flickering_cat_element 8181.0
best_flickering_cat_log_idx 94
BEST FLICKERING LOG:  bagfile_19-01-10_0744_8 8181.0
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: construct-post.. Skipping
No Best log found for category: construct-sign.. Skipping
best_flickering_cat_element 1.0
best_flickering_cat_log_idx 61
BEST FLICKERING LOG:  bagfile_18-10-24_1129_2 1.0
best_flickering_cat_element 6970.0
best_flickering_cat_log_idx 53
BEST FLICKERING LOG:  bagfile_18-10-06_1113_21 6970.0
best_flickering_cat_element 9.0
best_flickering_cat_log_idx 42
BEST FLICKERING LOG:  alanturing-2019-04-17--19-24-17--cam1 9.0
best_flickering_cat_element 575.0
best_flickering_

best_tracked_cat_element 23.0
best_tracked_cat_log_idx 17
BEST TRACKED LOG:  alanturing-2019-04-10--16-58-59 23.0
best_flickering_cat_element 27.0
best_flickering_cat_log_idx 15
BEST FLICKERING LOG:  alanturing-2019-04-10--16-20-59--cam2 27.0
best_flickering_cat_element 7634.0
best_flickering_cat_log_idx 38
BEST FLICKERING LOG:  daisy-2019-05-30--09-57-31--cam1 7634.0
No Best log found for category: construct-equipment.. Skipping
No Best log found for category: construct-pole.. Skipping
No Best log found for category: construct-post.. Skipping
No Best log found for category: construct-sign.. Skipping
No Best log found for category: motorcycle.. Skipping
best_tracked_cat_element 6.0
best_tracked_cat_log_idx 3
BEST TRACKED LOG:  alanturing-2019-02-14--16-33-24--cam1 6.0
best_tracked_cat_element 1.0
best_tracked_cat_log_idx 14
BEST TRACKED LOG:  alanturing-2019-04-10--16-55-59--cam1 1.0
best_tracked_cat_element 179.0
best_tracked_cat_log_idx 13
BEST TRACKED LOG:  alanturing-2019-04-09--11

In [102]:
import urllib
import ntpath
import json
from pipelineapi import PipelineAPI
import os

def get_dataset(video_source_names = [], kache_anns_dir = os.getcwd()):
    data_pipeline_api = PipelineAPI("http://ec2-54-149-236-75.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')

    batch_size = 0
    batch = 0
    batch_anns = []


    dataset_names = ['032019']
    

    databunch = data_pipeline_api.data(dataset_names=dataset_names, video_source_names=video_source_names, categories_names=[],
                    label_statuses=[], page_size=None, frame_id=None) # returns generator

    for anns in databunch:
        anns = update_traffic_light_cats([anns])
        batch_anns.extend(anns)
        batch_size += len(anns)
        if batch_size % 5000 == 0:
            with open(os.path.join(kache_anns_dir,f'{"^".join(dataset_names)}_{batch}--flickers.json'), 'w') as fn:
                json.dump(batch_anns, fn)
            batch_anns = []
            batch += 1

    # Save any remaining frames
    if len(batch_anns) != 0:
        with open(os.path.join(kache_anns_dir,f'{"^".join(dataset_names)}_{batch}--flickers.json'), 'w') as fn:
            json.dump(batch_anns, fn)


In [ ]:
import random
import glob
import json
import numpy as np
import hashlib
import toml
import shutil
import subprocess
from subprocess import Popen,PIPE,STDOUT,call


anns_dir = '/media/kuser/Data/06242019_trn/'


dataset_files = glob.glob(os.path.join(anns_dir, '*.json'))
dataset_stats = {}

for idx, ann_file in enumerate(dataset_files):
    with open(ann_file, 'r') as f:
        dataset = json.load(f)
        dataset_stats[idx] = len(dataset)
        
print(dataset_stats)
tot_frames = np.sum(list(dataset_stats.values()))
print(tot_frames)

remove_count = tot_frames - 30000 # labeling budget (30k in production)

if remove_count > 0:
    databatch_portions = [float(datalen/tot_frames) for datalen in dataset_stats.values()]
    remove_counts = [int(remove_count*portion)+1 for portion in databatch_portions]
    
    # Randomly remove frames based on remove count
    for idx, ann_file in enumerate(dataset_files):
        with open(ann_file, 'r') as f:
            dataset = json.load(f)
        frames = list(range(len(dataset)))
        random.shuffle(frames)
        deleted_frames = frames[:remove_counts[idx]]
        dataset = [img_data for idx, img_data in enumerate(dataset) if  idx not in deleted_frames]
        with open(ann_file, 'w+') as f:
            json.dump(dataset, f)
            

# Change name/url to s3_url
for idx, ann_file in enumerate(dataset_files):
    with open(ann_file, 'r') as f:
        dataset = json.load(f)

    for img_data in dataset:
        hex_val = os.path.abspath(img_data['url']).replace('/', '^^')
        hash_object = hashlib.sha1(str.encode(hex_val))
        hex_dig = hash_object.hexdigest()

        tmp_img_dir = os.path.join(anns_dir, 'images')
        os.makedirs(tmp_img_dir, exist_ok = True)
        orig_frame_path = img_data['url']
        with open('/media/kuser/Data/nebula/pipelines/configs/entanglement_cfg.tml', 'r') as tmlfile:
            cfg_data = toml.load(tmlfile)

        orig_frame_path = os.path.join(cfg_data['entanglement']['master_nebula_dir'], img_data['url'].split('nebula')[-1].strip('/') )
        new_path = os.path.join(tmp_img_dir, str(hex_dig)+os.path.splitext(path_leaf(img_data['url'])[-1]))

        shutil.copyfile(orig_frame_path, new_path)
        s3uri = send_to_s3( new_path )

        img_data['url'] = s3uri



    with open(ann_file, 'w+') as f:
        json.dump(dataset, f)
    
    